In [23]:
import random
import time
import statistics

In [24]:
POP_SIZE = 20
GENES_PER_QUEEN = 3
NUM_QUEENS = 8
CHROMOSOME_SIZE = NUM_QUEENS * GENES_PER_QUEEN
CROSSOVER_RATE = 0.8
MUTATION_RATE = 0.03
MAX_GENERATIONS = 1000
RUNS = 50

In [25]:
def decode(individual):
    """Converte a string de bits em uma lista de inteiros."""
    return [int(individual[i:i+3], 2) for i in range(0, len(individual), 3)]

def count_attacks(board):
    """Calcula o número de pares de rainhas que se atacam."""
    attacks = 0
    n = len(board)
    for i in range(n):
        for j in range(i + 1, n):
            if board[i] == board[j] or abs(board[i] - board[j]) == abs(i - j):
                attacks += 1
    return attacks

def fitness_ga(individual):
    """Função de aptidão."""
    attacks = count_attacks(decode(individual))
    return 28 - attacks

def roulette_selection(population, fitnesses):
    total_fitness = sum(fitnesses)
    if total_fitness == 0: return random.choice(population)
    pick = random.uniform(0, total_fitness)
    current = 0
    for ind, fit in zip(population, fitnesses):
        current += fit
        if current > pick:
            return ind
    return population[-1]

def crossover(p1, p2):
    if random.random() < CROSSOVER_RATE:
        point = random.randint(1, CHROMOSOME_SIZE - 1)
        return p1[:point] + p2[point:], p2[:point] + p1[point:]
    return p1, p2

def mutate(individual):
    ind_list = list(individual)
    for i in range(len(ind_list)):
        if random.random() < MUTATION_RATE:
            ind_list[i] = '1' if ind_list[i] == '0' else '0'
    return "".join(ind_list)

def run_ga():
    start_time = time.time()
    population = [''.join(random.choice('01') for _ in range(CHROMOSOME_SIZE)) for _ in range(POP_SIZE)]

    for gen in range(MAX_GENERATIONS):
        fitnesses = [fitness_ga(ind) for ind in population]

        if 28 in fitnesses:
            best_ind = population[fitnesses.index(28)]
            duration = time.time() - start_time
            return decode(best_ind), 0, gen + 1, duration

        new_pop = []
        best_idx = np.argmax(fitnesses)
        new_pop.append(population[best_idx])

        while len(new_pop) < POP_SIZE:
            p1 = roulette_selection(population, fitnesses)
            p2 = roulette_selection(population, fitnesses)
            c1, c2 = crossover(p1, p2)
            new_pop.append(mutate(c1))
            if len(new_pop) < POP_SIZE:
                new_pop.append(mutate(c2))
        population = new_pop

    fitnesses = [fitness_ga(ind) for ind in population]
    best_idx = fitnesses.index(max(fitnesses))
    best_board = decode(population[best_idx])
    duration = time.time() - start_time
    return best_board, count_attacks(best_board), MAX_GENERATIONS, duration

In [26]:
all_iters = []
all_times = []
distinct_solutions = {}

for _ in range(RUNS):
    board, h, iters, duration = run_ga()
    all_iters.append(iters)
    all_times.append(duration)

    board_tuple = tuple(board)
    if board_tuple not in distinct_solutions:
        distinct_solutions[board_tuple] = h

In [27]:
mean_it = statistics.mean(all_iters)
std_it = statistics.stdev(all_iters)
mean_ti = statistics.mean(all_times)
std_ti = statistics.stdev(all_times)

print(f"{'='*40}")
print(f"ALGORITMO GENÉTICO - RESULTADOS")
print(f"Gerações até parar: Média = {mean_it:.2f} | Desvio Padrão = {std_it:.2f}")
print(f"Tempo de execução:  Média = {mean_ti:.4f}s | Desvio Padrão = {std_ti:.4f}s")
print(f"{'='*40}\n")

ALGORITMO GENÉTICO - RESULTADOS
Gerações até parar: Média = 559.98 | Desvio Padrão = 409.49
Tempo de execução:  Média = 0.1572s | Desvio Padrão = 0.1191s



In [28]:
# 5 melhores soluções distintas
sorted_solutions = sorted(distinct_solutions.items(), key=lambda x: x[1])
top_5 = sorted_solutions[:5]

print(f"5 melhores soluções distintas encontradas:")
for i, (board, h) in enumerate(top_5, 1):
    print(f"\n{i}ª Solução (Ataques: {h}) - Configuração: {list(board)}")
    for r in range(8):
        row_str = "".join(" Q " if board[c] == r else " . " for c in range(8))
        print(row_str)

5 melhores soluções distintas encontradas:

1ª Solução (Ataques: 0) - Configuração: [4, 0, 7, 3, 1, 6, 2, 5]
 .  Q  .  .  .  .  .  . 
 .  .  .  .  Q  .  .  . 
 .  .  .  .  .  .  Q  . 
 .  .  .  Q  .  .  .  . 
 Q  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  Q 
 .  .  .  .  .  Q  .  . 
 .  .  Q  .  .  .  .  . 

2ª Solução (Ataques: 0) - Configuração: [5, 2, 4, 7, 0, 3, 1, 6]
 .  .  .  .  Q  .  .  . 
 .  .  .  .  .  .  Q  . 
 .  Q  .  .  .  .  .  . 
 .  .  .  .  .  Q  .  . 
 .  .  Q  .  .  .  .  . 
 Q  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  Q 
 .  .  .  Q  .  .  .  . 

3ª Solução (Ataques: 0) - Configuração: [1, 5, 0, 6, 3, 7, 2, 4]
 .  .  Q  .  .  .  .  . 
 Q  .  .  .  .  .  .  . 
 .  .  .  .  .  .  Q  . 
 .  .  .  .  Q  .  .  . 
 .  .  .  .  .  .  .  Q 
 .  Q  .  .  .  .  .  . 
 .  .  .  Q  .  .  .  . 
 .  .  .  .  .  Q  .  . 

4ª Solução (Ataques: 0) - Configuração: [6, 4, 2, 0, 5, 7, 1, 3]
 .  .  .  Q  .  .  .  . 
 .  .  .  .  .  .  Q  . 
 .  .  Q  .  .  .  .  . 
 .  .  .  .  .  . 